Your markdown or comments here

In [1]:
# -*- coding: utf-8 -*-
"""embedding-crawler.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1veoXGbqXGOxHJrYH6ZehSCyoLBhXvML9
"""

# for embedding.py file
# python3 -m venv venv
# source venv/bin/activate
# pip install python-dotenv langchain_openai langchain_community chromadb youtube-transcript-api pytube pypdf web3 SpeechRecognition opencv-python
# python3 embedding.py

%pip install --upgrade pip -q
%pip install python-dotenv langchain_openai langchain_community chromadb youtube-transcript-api pytube pypdf web3 -q
%pip install youtube-transcript-api bs4 pypdf -q
%pip install SpeechRecognition -q
%pip install opencv-python -q
%pip install beautifulsoup4 -q
%pip install python-pptx -q
%pip install 'unstructured[all]' -q

## For mac
# brew install libmagic
# brew install file

## For Ubuntu
# sudo apt-get update
# sudo apt-get install libmagic1 libmagic-dev


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:

# --- Step 1: 
from dotenv import load_dotenv
import os
import time
import requests
import yt_dlp
from bs4 import BeautifulSoup

from langchain_community.document_loaders import PyPDFLoader, YoutubeLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredPowerPointLoader


In [3]:
# --- Step 2: Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [10]:
# --- Step 3: Web crawler
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; FidesCrawler/1.0)"
}

web_urls = [
    "https://www.fidesinnova.io/",
    "https://fidesinnova.io/devices/",
    "https://fidesinnova.io/Contacts/",
    "https://fidesinnova.io/courses/",
    "https://linktr.ee/fidesinnova/",
    "https://play.google.com/store/apps/details?id=io.fidesinnova.front&pli=1/",
    "https://fidesinnova.io/blog-standard/",
    "https://fidesinnova.io/About/",
    "https://fidesinnova.io/#articles/",
    "https://discord.com/invite/NQdM6JGwcs/",
    "https://fidesinnova.io/service-market/",
    "https://fidesinnova.io/service-contract-2/",
    "https://fidesinnova.io/d2pos/",
    "https://fidesinnova.io/web3/",
    "https://fidesinnova.io/service-contract/",
    "https://fidesinnova.io/miotn/",
    "https://fidesinnova.io/consensus-algorithms/",
    "https://fidesinnova.io/mqtt-mqtts/",
    "https://explorer.fidesinnova.io/",
    "https://apps.apple.com/ca/app/fidesinnova/id6477492757/",
    "https://www.youtube.com/@FidesInnova/playlists/",
    "https://x.com/FidesInnova/"

]

def crawl_web_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else "No Title"
        text = soup.get_text(separator='\n', strip=True)
        return Document(page_content=text, metadata={"source": url, "title": title, "type": "Web"})
    except Exception as e:
        print(f"Error crawling {url}: {e}")
        return None

web_docs = [doc for url in web_urls if (doc := crawl_web_url(url))]
print("=== ALL web_docs ===)")
print(len(web_docs))



Error crawling https://x.com/FidesInnova/: 400 Client Error: Bad Request for url: https://x.com/FidesInnova/
=== ALL web_docs ===)
21


In [11]:
# --- Step 4: GitHub Crawler ---
github_urls = [
    "https://github.com/TheArchitect2000/iot-server/",
    "https://github.com/TheArchitect2000/zkiot-riscv-qemu-c/",
    "https://github.com/TheArchitect2000/Fides-Innova-WiKi/",
    "https://github.com/TheArchitect2000/zkiot-arm-siemens-iot2050-c/",
    "https://github.com/TheArchitect2000/fidesinnova-smart-contract-protocol/",
    "https://github.com/TheArchitect2000/zkp-explorer/",
    "https://github.com/TheArchitect2000/Fides-LLM/",
    "https://github.com/TheArchitect2000/Verifiable-Computing-AI-Agents/",
    "https://github.com/TheArchitect2000/evm-server/",
    "https://github.com/TheArchitect2000/New-IoT-Device-Integration/",
    "https://github.com/TheArchitect2000/zkiot-riscv-qemu-rust/"
]

def crawl_github_url(url):
    
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
                    "(KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else "No Title"
        text = soup.get_text(separator='\n', strip=True)
        return Document(page_content=text, metadata={"source": url, "title": title, "type": "Web"})
    except Exception as e:
        print(f"Error crawling GitHub {url}: {e}")
        return None

github_docs = [doc for url in github_urls if (doc := crawl_github_url(url))]

print("=== ALL github_docs ===)")
print(len(github_docs))



=== ALL github_docs ===)
11


In [12]:
# --- Step 5: YouTube Channel Crawler ---

# Fides Innova YouTube Channel ID
CHANNEL_ID = "UCrrqGYx98H1dPdZsNb1i9-g"
CHANNEL_URL = f"https://www.youtube.com/channel/{CHANNEL_ID}"

def fetch_video_urls(channel_url: str):
    ydl_opts = {
        'ignoreerrors': True,
        'quiet': True,
        'extract_flat': True,  # Only metadata, not downloading
        'force_generic_extractor': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(channel_url, download=False)
        video_urls = []
        video_ids = []

        if 'entries' in result:
            for entry in result['entries']:
                if entry and 'id' in entry:
                    video_url = f"https://www.youtube.com/watch?v={entry['id']}"
                    video_urls.append(video_url)
                    video_ids.append(entry['id'])
        return video_ids

# Fetch and display video URLs
video_ids = fetch_video_urls(CHANNEL_URL)
print(f"✅ Found {len(video_ids)} videos on channel.\n")

youtube_docs = []
second_try_idx = []
third_try_idx = []
for idx in video_ids:
    a = YoutubeLoader(idx)
    try:
        youtube_docs.extend(a.load())
        print(idx + " is loaded.")
    except:
        print(f"{idx} is not loaded.")
        second_try_idx.append(str(idx))
        print("\n unloaded list ")
        print(second_try_idx)
        print("\n")

print("\n")
print("second try for unloded videos.")

for idx in second_try_idx:
    print("waiting 10 seconds...")
    time.sleep(10)
    a = YoutubeLoader(idx)
    try:
        youtube_docs.extend(a.load())
        print(idx + " is loaded in the second try.")
    except:
        print("\n")
        print(f"{idx} is not loaded in the second try.")
        third_try_idx.append(str(idx))
        print("\n unloaded list for the second time ")
        print(second_try_idx)
        print("\n")

def change_YouTube_doc(doc):
    doc.metadata['type']='YouTube'
    return doc

youtube_docs = list(map(change_YouTube_doc, youtube_docs))

print("=== ALL youtube videos ===)")
print(len(youtube_docs))
print("=== Loaded youtube videos ===)")
print(len(youtube_docs)-len(third_try_idx))


✅ Found 29 videos on channel.

kgYxyxeDNl4 is loaded.
VZMBE2NLSC4 is loaded.
FtELZXp3qmQ is loaded.
loHb12HmJas is loaded.
YpfFHI3Ivmo is loaded.
sPWmDRItCNk is loaded.
1DAkWOP_uY4 is loaded.
2R_bDZ0sasM is loaded.
3GVfyu4uzhs is loaded.
c8dQtxByOcY is not loaded.

 unloaded list 
['c8dQtxByOcY']


RcaMNALojIo is loaded.
qMjLWztjqvk is not loaded.

 unloaded list 
['c8dQtxByOcY', 'qMjLWztjqvk']


DzvQhVw0tNU is loaded.
asw_A0VSObI is loaded.
dBEEPLjEJGU is loaded.
s1ZPMrqU0c0 is not loaded.

 unloaded list 
['c8dQtxByOcY', 'qMjLWztjqvk', 's1ZPMrqU0c0']


K_7RhSkWgbM is loaded.
92QUILNAhDI is loaded.
WZnxV-f6E7k is loaded.
tNWEXETid7Q is not loaded.

 unloaded list 
['c8dQtxByOcY', 'qMjLWztjqvk', 's1ZPMrqU0c0', 'tNWEXETid7Q']


8Jk19GqcEW4 is loaded.
Rfl3rjQor-o is not loaded.

 unloaded list 
['c8dQtxByOcY', 'qMjLWztjqvk', 's1ZPMrqU0c0', 'tNWEXETid7Q', 'Rfl3rjQor-o']


zPfHYADmUGY is not loaded.

 unloaded list 
['c8dQtxByOcY', 'qMjLWztjqvk', 's1ZPMrqU0c0', 'tNWEXETid7Q', 'Rfl3rjQor-o'

In [13]:
# --- Step 6: Load PDFs ---
pdf_docs = []
pdf_files = [
    "PDF/zkIoT.pdf",
    "PDF/Fides Innova Company Profile 2025.pdf",
    "PDF/Fides Innova Pitch Deck - v10.pdf"
]

for path in pdf_files:
    try:
        loader = PyPDFLoader(path)
        pdf_docs.extend(loader.load())
        print(len(pdf_docs))
    except Exception as e:
        print(f"Error loading PDF {path}: {e}")

def change_pdf_doc(doc):
    doc.metadata['type']='PDF'
    return doc

pdf_docs = list(map(change_pdf_doc, pdf_docs))

print("=== ALL PDF docs ===)")
print(len(pdf_docs))

7
21
37
=== ALL PDF docs ===)
37


In [14]:

pptx_docs = []
pptx_files = [
    "PPTX/FidesinnovaDeck-v11.pptx"
]

for path in pptx_files:
    try:
        loader = UnstructuredPowerPointLoader(path)
        pptx_docs.extend(loader.load())
        print(f"✅ Loaded: {path}")
    except Exception as e:
        print(f"❌ Error loading PPTX {path}: {e}")

# Add metadata
def change_pptx_doc(doc):
    doc.metadata['type'] = 'PPTX'
    return doc

pptx_docs = list(map(change_pptx_doc, pptx_docs))

print("=== ALL PPTX docs ===)")
print(len(pptx_docs))

✅ Loaded: PPTX/FidesinnovaDeck-v11.pptx
=== ALL PPTX docs ===)
1


In [16]:
print(pptx_docs)

[Document(metadata={'source': 'PPTX/FidesinnovaDeck-v11.pptx', 'type': 'PPTX'}, page_content="FIDES INNOVA\n\nVerifiable Computing Ecosystem\n\nW W W . F I D E S I N N O V A . I O\n\n\n\nFIDES INNOVA LABS\n\nThe Future is Verifiable\n\nA pioneer in enabling secure computation across IoT and ML \n\nRevolutionizing trust by introducing Agent-based Verifiable Computing \n\nleveraging zero-knowledge proof (ZKP), DePIN and Agentic AI. \n\n\n\nFOUNDERS\n\nMel McCann\n\nCo-founder\n\nReza Ramezan, Ph.D.\n\nCo-Founder\n\nFormer Research Lead at Cardano Foundation\n\nWith over 20 years of executive experience at leading firms like AT&T, Huawei, Telus, and Cardano Foundation, Reza is a proven innovator and entrepreneur. He holds a Ph.D. from the University of British Columbia and has numerous patents and publications in blockchain, zero- knowledge proofs, and IoT.\n\nHamed Shah-Mansouri, Ph.D.\n\nCo-founder\n\nWith over 10 years of research and development experience, Hamed is a C-level executiv

In [ ]:
# --- Step 7: Split & Vectorize ---
all_docs = web_docs + github_docs + youtube_docs + pdf_docs + pptx_docs

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
split_docs = splitter.split_documents(all_docs)

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    collection_name="fides_crawled_data",
    embedding_function=embedding,
    persist_directory="./chroma_langchain_db"
)

vectordb.add_documents(split_docs)
print("✅ All documents crawled, split, and stored in vector DB.")

print("=== ALL Doc.s ===)")
print(len(all_docs))

✅ All documents crawled, split, and stored in vector DB.
=== ALL Doc.s ===)
99


In [18]:
len(vectordb)

10103